In [1]:
import os
import re
import json
import time
import librosa
import subprocess
import shlex
import torch
import fairseq
import numpy as np
import soundfile as sf

import numpy as np
import codecs
import os

from tqdm import tqdm

if 'cd' not in globals():
    os.chdir('..')
    cd = True


In [2]:

babel_langs="103 107 206 307 402 404 505".split()
babel_recog="101 203".split()
gp_langs="Czech French Mandarin Thai Bulgarian German Turkish Portuguese".split()
gp_recog="Spanish Polish Croatian".split()

symbol_dict = {}
for babel_lang in babel_langs + babel_recog:
    tr_dir = os.path.join('data', babel_lang, 'data', 'train_' + babel_lang, 'text')
    dev_dir = os.path.join('data', babel_lang, 'data', 'dev_' + babel_lang, 'text')
    eval_dir = os.path.join('data', babel_lang, 'data', 'eval_' + babel_lang, 'text')
    
    all_txt = []
    for text_dir in [tr_dir, dev_dir, eval_dir]:
        f = codecs.open(text_dir).readlines()
        for line in f:
            true_text = line.split(maxsplit = 1)
            if len(true_text) != 1:
                true_text = true_text[1].replace('\n','')
                if '<' in true_text:
                    print(babel_lang)
                    print('Damn!')
                    print(true_text)
                all_txt.append(true_text)
            
    all_txt = ''.join(all_txt)
    labels = list(set(all_txt))
    
    symbol_dict[babel_lang] = sorted(labels)
    

In [4]:

for gp_lang in gp_langs + gp_recog:
    tr_dir = os.path.join('data', 'GlobalPhone/gp_' + gp_lang + '_train', 'text')
    dev_dir = os.path.join('data', 'GlobalPhone/gp_' + gp_lang + '_dev', 'text')
    eval_dir = os.path.join('data', 'GlobalPhone/gp_' + gp_lang + '_eval', 'text')
    
    all_txt = []
    for text_dir in [tr_dir, dev_dir, eval_dir]:
        f = codecs.open(text_dir).readlines()
        for line in f:
            true_text = line.split(maxsplit = 1)[1].replace('\n','')
            if '<' in true_text:
                print('Damn!')
                print(true_text)
            all_txt.append(true_text)
            
    all_txt = ''.join(all_txt)
    labels = list(set(all_txt))
    
    symbol_dict[gp_lang] = sorted(labels)

In [ ]:
'103', '107', '206', '307', '402', '404', 'BG', 'CH', 'CZ', 'FR', 'GE', 'N', 'PO', 'TH', 'TU'

In [7]:
len(symbol_dict),  symbol_dict.keys()

(20,
 dict_keys(['103', '107', '206', '307', '402', '404', '505', '101', '203', 'Czech', 'French', 'Mandarin', 'Thai', 'Bulgarian', 'German', 'Turkish', 'Portuguese', 'Spanish', 'Polish', 'Croatian']))

In [12]:
lang2ll = {
    '103': '103', 
    '107': '107',
    '206': '206',
    '307': '307',
    '402': '402',
    '404': '404',
    '101': '101',
    '203': '203',
    'Bulgarian': 'BG',
    'Mandarin': 'CH',
    'Czech': 'CZ',
    'Croatian': 'CR',
    'French': 'FR',
    'German': 'GE',
    '505': 'N',
    'Polish': 'PL',
    'Portuguese': 'PO',
    'Spanish': 'SP',
    'Thai': 'TH',
    'Turkish': 'TU'
}

In [13]:
len(lang2ll)

20

In [15]:
lang2ph = {}
for l, ph in symbol_dict.items():
    ll = lang2ll[l]
    lang2ph[ll] = [p for p in ph if p != ' ']

In [18]:
with open('phones/lang2ph.json', 'w') as f:
    json.dump(lang2ph, f, indent=4)